In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook810eb1472d/_output_.zip
/kaggle/input/notebook810eb1472d/__results__.html
/kaggle/input/notebook810eb1472d/__notebook__.ipynb
/kaggle/input/notebook810eb1472d/__output__.json
/kaggle/input/notebook810eb1472d/custom.css
/kaggle/input/notebook810eb1472d/__results___files/__results___2_0.jpg
/kaggle/input/notebook810eb1472d/__results___files/__results___2_0.png


In [2]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = "/kaggle/input/notebook810eb1472d/_output_.zip"
extract_path = "/kaggle/working/frames"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check contents
os.listdir(extract_path)


['__pycache__', 'combined_dataset.csv', 'frames']

In [ ]:
# import os
# import time
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report, confusion_matrix
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import seaborn as sns
# import joblib

# # -------------------------
# # Configurations
# # -------------------------
# csv_path = "/kaggle/working/frames/combined_dataset.csv"
# base_dir = "/kaggle/working/split_data"
# os.makedirs(base_dir, exist_ok=True)
# tabular_cols = ['EAR', 'MAR', 'Head_Tilt', 'Head_Nod']
# batch_size = 32
# epochs = 20
# timeout_seconds = 11 * 3600 + 45 * 60
# start_time = time.time()
# timeout_reached = False

# # -------------------------
# # Load CSV & Clean
# # -------------------------
# df = pd.read_csv(csv_path)
# df['img_path'] = df['img_path'].apply(lambda x: os.path.join("/kaggle/working/frames/frames", os.path.basename(x)))
# df['Label'] = df['Label'].map({0: 0, 5: 1, 10: 2})
# df[tabular_cols] = df[tabular_cols].replace([np.inf, -np.inf], np.nan)
# df = df.dropna(subset=tabular_cols).reset_index(drop=True)

# scaler = StandardScaler()
# df[tabular_cols] = scaler.fit_transform(df[tabular_cols])
# joblib.dump(scaler, "/kaggle/working/tabular_scaler.pkl")  # Save scaler for later inference

# # -------------------------
# # Split Dataset & Save
# # -------------------------
# def save_split_data(split_df, name):
#     folder = os.path.join(base_dir, name)
#     os.makedirs(folder, exist_ok=True)
#     split_df.to_csv(os.path.join(folder, "dataset.csv"), index=False)

# train_val_df, test_df = train_test_split(df, test_size=0.15, stratify=df['Label'], random_state=42)
# train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'], random_state=42)

# save_split_data(train_df, "train")
# save_split_data(val_df, "val")
# save_split_data(test_df, "test")

# # -------------------------
# # Dataset Class
# # -------------------------
# class DrowsinessCombinedDataset(Dataset):
#     def __init__(self, dataframe, transform=None):
#         self.df = dataframe.reset_index(drop=True)
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         image = Image.open(row['img_path']).convert('RGB')
#         if self.transform:
#             image = self.transform(image)
#         tabular = torch.tensor(row[tabular_cols].astype(np.float32).values, dtype=torch.float32)
#         label = torch.tensor(int(row['Label'])).long()
#         return image, tabular, label

# # -------------------------
# # Transforms & Dataloaders
# # -------------------------
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# train_loader = DataLoader(DrowsinessCombinedDataset(train_df, transform), batch_size=batch_size, shuffle=True, num_workers=2)
# val_loader = DataLoader(DrowsinessCombinedDataset(val_df, transform), batch_size=batch_size, num_workers=2)
# test_loader = DataLoader(DrowsinessCombinedDataset(test_df, transform), batch_size=batch_size, num_workers=2)

# # -------------------------
# # Model Definition
# # -------------------------
# class DepthwiseSeparableConv(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1):
#         super().__init__()
#         self.depthwise = nn.Conv2d(in_channels, in_channels, 3, stride, 1, groups=in_channels, bias=False)
#         self.pointwise = nn.Conv2d(in_channels, out_channels, 1, bias=False)
#         self.bn = nn.BatchNorm2d(out_channels)
#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, x):
#         x = self.depthwise(x)
#         x = self.pointwise(x)
#         x = self.bn(x)
#         return self.relu(x)

# class MobileNetWithTabularLSTM(nn.Module):
#     def __init__(self, num_tabular_features, num_classes=3):
#         super().__init__()
#         self.cnn = nn.Sequential(
#             nn.Conv2d(3, 32, 3, stride=2, padding=1, bias=False),
#             nn.BatchNorm2d(32),
#             nn.ReLU(inplace=True),
#             DepthwiseSeparableConv(32, 64, 1),
#             DepthwiseSeparableConv(64, 128, 2),
#             DepthwiseSeparableConv(128, 128, 1),
#             DepthwiseSeparableConv(128, 256, 2),
#             DepthwiseSeparableConv(256, 256, 1),
#             nn.AdaptiveAvgPool2d(1)
#         )
#         self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(128 + num_tabular_features, 128),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(128, num_classes)
#         )

#     def forward(self, image, tabular):
#         x_img = self.cnn(image)
#         x_img = torch.flatten(x_img, 1).unsqueeze(1)
#         x_lstm, _ = self.lstm(x_img)
#         x_lstm = x_lstm[:, -1, :]
#         x = torch.cat([x_lstm, tabular], dim=1)
#         return self.fc(x)

# # -------------------------
# # Training Setup
# # -------------------------
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MobileNetWithTabularLSTM(num_tabular_features=len(tabular_cols)).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# checkpoint_path = "/kaggle/working/checkpoint.pth"
# failsafe_model_path = "/kaggle/working/failsafe_model.pth"
# best_model_path = "/kaggle/working/best_model.pth"
# start_epoch = 0
# best_val_acc = 0.0
# train_accuracies, val_accuracies = [], []

# # -------------------------
# # Resume Checkpoint
# # -------------------------
# if os.path.exists(checkpoint_path):
#     checkpoint = torch.load(checkpoint_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
#     start_epoch = checkpoint['epoch'] + 1
#     best_val_acc = checkpoint['best_val_acc']
#     print(f"✅ Resumed from epoch {start_epoch}")

# # -------------------------
# # Training Loop
# # -------------------------
# for epoch in range(start_epoch, epochs):
#     model.train()
#     correct, total, total_loss = 0, 0, 0
#     for images, tabular, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
#         if time.time() - start_time >= timeout_seconds:
#             timeout_reached = True
#             torch.save(model.state_dict(), failsafe_model_path)
#             print("⏰ Timeout reached. Saving failsafe model.")
#             break
#         images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images, tabular)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#         preds = outputs.argmax(1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     if timeout_reached: break
#     train_acc = 100 * correct / total
#     train_accuracies.append(train_acc)
#     print(f"Train Loss: {total_loss:.4f}, Accuracy: {train_acc:.2f}%")

#     model.eval()
#     val_correct, val_total = 0, 0
#     with torch.no_grad():
#         for images, tabular, labels in val_loader:
#             if time.time() - start_time >= timeout_seconds:
#                 timeout_reached = True
#                 torch.save(model.state_dict(), failsafe_model_path)
#                 print("⏰ Timeout during validation.")
#                 break
#             images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
#             outputs = model(images, tabular)
#             val_correct += (outputs.argmax(1) == labels).sum().item()
#             val_total += labels.size(0)

#     if timeout_reached: break
#     val_acc = 100 * val_correct / val_total
#     val_accuracies.append(val_acc)
#     print(f"Validation Accuracy: {val_acc:.2f}%")

#     scheduler.step()
#     torch.save({
#         'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'scheduler_state_dict': scheduler.state_dict(),
#         'best_val_acc': best_val_acc
#     }, checkpoint_path)

#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model.state_dict(), best_model_path)
#         print("✅ Best model saved.")

# # -------------------------
# # Evaluation and Plots
# # -------------------------
# if not timeout_reached:
#     model.load_state_dict(torch.load(best_model_path))
#     model.eval()
#     all_preds, all_labels = [], []
#     with torch.no_grad():
#         for images, tabular, labels in test_loader:
#             images, tabular = images.to(device), tabular.to(device)
#             outputs = model(images, tabular)
#             all_preds.extend(outputs.argmax(1).cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     test_accuracy = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
#     print("\n--- Test Set Performance ---")
#     print(classification_report(all_labels, all_preds, target_names=["Alert (0)", "Slightly Drowsy (1)", "Drowsy (2)"]))
#     print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")

#     cm = confusion_matrix(all_labels, all_preds)
#     plt.figure(figsize=(6, 5))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Alert", "Slightly Drowsy", "Drowsy"], yticklabels=["Alert", "Slightly Drowsy", "Drowsy"])
#     plt.xlabel("Predicted")
#     plt.ylabel("True")
#     plt.title("Confusion Matrix")
#     plt.savefig("/kaggle/working/confusion_matrix.png")
#     plt.show()

#     plt.figure(figsize=(8, 5))
#     plt.plot(range(start_epoch+1, epochs+1), train_accuracies, label='Train Accuracy')
#     plt.plot(range(start_epoch+1, epochs+1), val_accuracies, label='Validation Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy (%)')
#     plt.title('Training vs Validation Accuracy')
#     plt.legend()
#     plt.grid(True)
#     plt.savefig("/kaggle/working/accuracy_plot.png")
#     plt.show()

In [3]:
import os, time, re
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------
# Configs
# -------------------------
csv_path = "/kaggle/working/frames/combined_dataset.csv"
frame_root = "/kaggle/working/frames/frames"
base_dir = "/kaggle/working/split_data"
seq_len = 5
tabular_cols = ['EAR', 'MAR', 'Head_Tilt', 'Head_Nod']
batch_size = 32
epochs = 4
timeout_seconds = 11 * 3600 + 45 * 60
start_time = time.time()
timeout_reached = False

os.makedirs(base_dir, exist_ok=True)

# -------------------------
# Load & Prepare Data
# -------------------------
df = pd.read_csv(csv_path)
df['img_path'] = df['img_path'].apply(lambda x: os.path.join(frame_root, os.path.basename(x)))
df['Label'] = df['Label'].map({0: 0, 5: 1, 10: 2})
df[tabular_cols] = df[tabular_cols].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=tabular_cols).reset_index(drop=True)

# Extract video_id and frame_number
def extract_metadata(path):
    match = re.search(r"/frames/(.+?)/frame_(\d+)", path)
    if match:
        return match.group(1), int(match.group(2))
    return "unknown", -1

df[['video_id', 'frame_number']] = df['img_path'].apply(lambda x: pd.Series(extract_metadata(x)))

# Normalize tabular features
scaler = StandardScaler()
df[tabular_cols] = scaler.fit_transform(df[tabular_cols])
joblib.dump(scaler, "/kaggle/working/tabular_scaler.pkl")

# -------------------------
# Build Sequences
# -------------------------
df = df.sort_values(['video_id', 'frame_number'])

sequence_data = []
for _, group in df.groupby('video_id'):
    group = group.reset_index(drop=True)
    for i in range(len(group) - seq_len + 1):
        window = group.iloc[i:i+seq_len]
        if len(window['Label'].unique()) == 1:
            sequence_data.append({
                'img_paths': window['img_path'].tolist(),
                'tabular': window[tabular_cols].values,
                'label': window['Label'].iloc[0]
            })

seq_df = pd.DataFrame(sequence_data)
train_val_df, test_df = train_test_split(seq_df, test_size=0.15, stratify=seq_df['label'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['label'], random_state=42)

# -------------------------
# Dataset & Dataloader
# -------------------------
class DrowsinessSequenceDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        images = [Image.open(p).convert("RGB") for p in item['img_paths']]
        if self.transform:
            images = [self.transform(img) for img in images]
        images = torch.stack(images)
        tabular = torch.tensor(item['tabular'], dtype=torch.float32)
        label = torch.tensor(item['label'], dtype=torch.long)
        return images, tabular, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(DrowsinessSequenceDataset(train_df, transform), batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(DrowsinessSequenceDataset(val_df, transform), batch_size=batch_size, num_workers=2)
test_loader = DataLoader(DrowsinessSequenceDataset(test_df, transform), batch_size=batch_size, num_workers=2)

# -------------------------
# Model
# -------------------------
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, 3, stride, 1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        return self.relu(x)

class MobileNetSequenceModel(nn.Module):
    def __init__(self, num_tabular_features, num_classes=3):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            DepthwiseSeparableConv(32, 64),
            DepthwiseSeparableConv(64, 128, 2),
            DepthwiseSeparableConv(128, 128),
            DepthwiseSeparableConv(128, 256, 2),
            DepthwiseSeparableConv(256, 256),
            nn.AdaptiveAvgPool2d(1)
        )
        self.lstm = nn.LSTM(input_size=256 + num_tabular_features, hidden_size=128, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, tabular):
        B, S, C, H, W = images.shape
        images = images.view(B*S, C, H, W)
        features = self.cnn(images).view(B, S, -1)
        x = torch.cat([features, tabular], dim=2)
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1])

# -------------------------
# Train Setup
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MobileNetSequenceModel(num_tabular_features=len(tabular_cols)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

checkpoint_path = "/kaggle/working/checkpoint.pth"
failsafe_model_path = "/kaggle/working/failsafe_model.pth"
best_model_path = "/kaggle/working/best_model.pth"
start_epoch = 0
best_val_acc = 0.0
train_accuracies, val_accuracies = [], []

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_val_acc = checkpoint['best_val_acc']
    print(f"✅ Resumed from epoch {start_epoch}")

# -------------------------
# Train Loop with ETA
# -------------------------
for epoch in range(start_epoch, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    model.train()
    correct, total, total_loss = 0, 0, 0

    progress_bar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}", ncols=100)
    for batch_idx, (images, tabular, labels) in enumerate(progress_bar, 1):
        if time.time() - start_time >= timeout_seconds:
            torch.save(model.state_dict(), failsafe_model_path)
            print("⏰ Timeout. Saved failsafe model.")
            timeout_reached = True
            break

        images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, tabular)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item()

        batch_acc = 100 * (preds == labels).sum().item() / labels.size(0)
        epoch_acc = 100 * correct / (total if total > 0 else 1)

        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Batch Acc': f'{batch_acc:.2f}%',
            'Epoch Acc': f'{epoch_acc:.2f}%'
        })

    if timeout_reached:
        break

    train_acc = 100 * correct / total
    train_accuracies.append(train_acc)
    print(f"\n✅ Train Loss: {total_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, tabular, labels in val_loader:
            images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
            outputs = model(images, tabular)
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_accuracies.append(val_acc)
    print(f"📊 Validation Accuracy: {val_acc:.2f}%")

    scheduler.step()
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_acc': best_val_acc
    }, checkpoint_path)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print("💾 Best model saved.")

# -------------------------
# Final Evaluation
# -------------------------
if not timeout_reached:
    model.load_state_dict(torch.load(best_model_path))
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, tabular, labels in test_loader:
            images, tabular = images.to(device), tabular.to(device)
            outputs = model(images, tabular)
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n--- Test Set Performance ---")
    print(classification_report(all_labels, all_preds, target_names=["Alert", "Slightly Drowsy", "Drowsy"]))
    print(f"✅ Test Accuracy: {np.mean(np.array(all_preds) == np.array(all_labels)) * 100:.2f}%")

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Alert", "Slightly Drowsy", "Drowsy"], yticklabels=["Alert", "Slightly Drowsy", "Drowsy"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.savefig("/kaggle/working/confusion_matrix.png")
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(range(start_epoch+1, epochs+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(start_epoch+1, epochs+1), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig("/kaggle/working/accuracy_plot.png")
    plt.show()



Epoch 1/6


Train Epoch 1: 100%|█| 16236/16236 [1:29:54<00:00,  3.01it/s, Loss=0.5229, Batch Acc=79.31%, Epoch A


✅ Train Loss: 11342.0216, Train Accuracy: 64.36%


📊 Validation Accuracy: 79.23%
💾 Best model saved.

Epoch 2/6


Train Epoch 2: 100%|█| 16236/16236 [1:28:45<00:00,  3.05it/s, Loss=0.3010, Batch Acc=82.76%, Epoch A

...


✅ Train Loss: 5252.9172, Train Accuracy: 86.17%


📊 Validation Accuracy: 87.88%
💾 Best model saved.

Epoch 4/6


Train Epoch 4: 100%|█| 16236/16236 [1:30:45<00:00,  2.98it/s, Loss=0.2906, Batch Acc=86.21%, Epoch A


✅ Train Loss: 4521.9855, Train Accuracy: 88.27%


KeyboardInterrupt: 

In [4]:
model.load_state_dict(torch.load(best_model_path))
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, tabular, labels in test_loader:
            images, tabular = images.to(device), tabular.to(device)
            outputs = model(images, tabular)
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n--- Test Set Performance ---")
    print(classification_report(all_labels, all_preds, target_names=["Alert", "Slightly Drowsy", "Drowsy"]))
    print(f"✅ Test Accuracy: {np.mean(np.array(all_preds) == np.array(all_labels)) * 100:.2f}%")

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Alert", "Slightly Drowsy", "Drowsy"], yticklabels=["Alert", "Slightly Drowsy", "Drowsy"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.savefig("/kaggle/working/confusion_matrix.png")
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(range(start_epoch+1, epochs+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(start_epoch+1, epochs+1), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig("/kaggle/working/accuracy_plot.png")
    plt.show()

IndentationError: unexpected indent (1286434559.py, line 2)